In [ ]:
from jdaviz.configs.mosviz.helper import MosViz

In [ ]:
mosviz = MosViz()
mosviz.app

The MOSViz parsers accepts lists of `Spectrum1D`, `SpectralCube`, and `CCDData` for 1D, 2D, and image data respectively. Alternatively, and the method used below, users can provide lists of file paths and MOSViz will internally attempt to parse them as their respective data type.

In [ ]:
import glob

spectra_1d = []
spectra_2d = []
images = []


for file_path in glob.glob("/Users/nmearl/Downloads/mosviz_nirspec_data_0.3/level3/*"):
    if 'x1d' in file_path:
        spectra_1d.append(file_path)
    elif 's2d' in file_path:
        spectra_2d.append(file_path)
        
        
for i in range(len(spectra_1d)):
    images.append("/Users/nmearl/Downloads/mosviz_nirspec_data_0.3/level3/mosviz_cutouts/227.fits")

mosviz.load_1d_spectra(spectra_1d)
mosviz.load_2d_spectra(spectra_2d)
mosviz.load_images(images)